In [30]:
import pandas as pd
import numpy as np

In [56]:
# Load datasets
transactions = pd.read_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/transactions_data.csv")
users = pd.read_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/users_data.csv")
cards = pd.read_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/cards_data.csv")

# Display first few rows
transactions.head(), users.head(), cards.head()


(        id                 date  client_id  card_id   amount  \
 0  7475327  2010-01-01 00:01:00       1556     2972  $-77.00   
 1  7475328  2010-01-01 00:02:00        561     4575   $14.57   
 2  7475329  2010-01-01 00:02:00       1129      102   $80.00   
 3  7475331  2010-01-01 00:05:00        430     2860  $200.00   
 4  7475332  2010-01-01 00:06:00        848     3915   $46.41   
 
             use_chip  merchant_id merchant_city merchant_state      zip   mcc  \
 0  Swipe Transaction        59935        Beulah             ND  58523.0  5499   
 1  Swipe Transaction        67570    Bettendorf             IA  52722.0  5311   
 2  Swipe Transaction        27092         Vista             CA  92084.0  4829   
 3  Swipe Transaction        27092   Crown Point             IN  46307.0  4829   
 4  Swipe Transaction        13051       Harwood             MD  20776.0  5813   
 
   errors  
 0    NaN  
 1    NaN  
 2    NaN  
 3    NaN  
 4    NaN  ,
      id  current_age  retirement_age  bi

In [47]:
# Quick data overview
transactions.info()
users.info()
cards.info()

# Check missing values
print("Transactions Missing:\n", transactions.isnull().sum())
print("Users Missing:\n", users.isnull().sum())
print("Cards Missing:\n", cards.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          float64
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.2+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month      

Handle Missing and Invalid Values
A. Transactions Table

In [ ]:
# ----------------------------------------
# 🔍 1. Handle duplicates
# ----------------------------------------
transactions.drop_duplicates(inplace=True)

# Convert 'date' to datetime
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')

# Convert IDs to string (in case they’re numeric but identifiers)
transactions['id'] = transactions['id'].astype(str)
transactions['client_id'] = transactions['client_id'].astype(str)
transactions['card_id'] = transactions['card_id'].astype(str)
transactions['merchant_id'] = transactions['merchant_id'].astype(str)

# ----------------------------------------
# 💰 2. Clean 'amount' column
# ----------------------------------------
transactions['amount'] = (
    transactions['amount']
    .astype(str)
    .str.replace('[^0-9.-]', '', regex=True)  # remove symbols/commas
)
transactions['amount'] = pd.to_numeric(transactions['amount'], errors='coerce')

transactions = transactions[transactions['amount'].notnull()]
transactions = transactions[transactions['amount'] > 0]

# ----------------------------------------
# 💳 3. Handle categorical columns
# ----------------------------------------
# Fill missing chip usage with "Unknown"
transactions['use_chip'] = transactions['use_chip'].fillna('Unknown')

# Fill missing merchant info
transactions['merchant_city'] = transactions['merchant_city'].fillna('Unknown')
transactions['merchant_state'] = transactions['merchant_state'].fillna('Unknown')

# Fill missing ZIP or MCC with default or placeholder
transactions['zip'] = transactions['zip'].fillna('00000')
transactions['mcc'] = transactions['mcc'].fillna('Unknown')

# ----------------------------------------
# ⚠️ 4. Handle 'errors' column
# ----------------------------------------
# Standardize text (lowercase, trim)
transactions['errors'] = (
    transactions['errors']
    .astype(str)
    .str.strip()
    .str.lower()
)

# Replace common invalid entries
transactions['errors'].replace(['nan', 'none', ''], np.nan, inplace=True)




/var/folders/2b/t1vzg7kd1y16_qk55mhrgf140000gn/T/ipykernel_75331/2054119757.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions['errors'].replace(['nan', 'none', ''], np.nan, inplace=True)


In [67]:
print("\nMissing values:\n", transactions.isnull().sum())
print("\nData types:\n", transactions.dtypes)
print("\nCleaned shape:", transactions.shape)



Missing values:
 id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                      0
mcc                      0
errors            12433446
dtype: int64

Data types:
 id                        object
date              datetime64[ns]
client_id                 object
card_id                   object
amount                   float64
use_chip                  object
merchant_id               object
merchant_city             object
merchant_state            object
zip                       object
mcc                        int64
errors                    object
dtype: object

Cleaned shape: (12635227, 12)


B. Users Table

In [72]:
# Step 1: Remove $ and commas, then convert to numeric
for col in ['yearly_income', 'total_debt', 'credit_score', 'per_capita_income']:
    users[col] = (
        users[col]
        .astype(str)                # ensure string type
        .str.replace(r'[\$,]', '', regex=True)  # remove $ and ,
        .astype(float)              # convert to float
    )

# Step 2: Fill missing numerical fields with median
num_cols = ['yearly_income', 'total_debt', 'credit_score', 'per_capita_income']
users[num_cols] = users[num_cols].fillna(users[num_cols].median())

# Step 3: Fill missing categorical fields with mode
users['gender'] = users['gender'].fillna(users['gender'].mode()[0])




C. Card table

In [78]:
import pandas as pd

# --- Remove duplicate card IDs ---
cards = cards.drop_duplicates(subset='id')

# --- Replace missing 'has_chip' or 'card_on_dark_web' flags ---
cards['has_chip'] = cards['has_chip'].fillna('No')
cards['card_on_dark_web'] = cards['card_on_dark_web'].fillna('No')

# --- Clean and convert 'credit_limit' ---
# Remove any $ or , and convert to float
cards['credit_limit'] = (
    cards['credit_limit']
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .astype(float)
)

# Fill missing credit limits with median
cards['credit_limit'] = cards['credit_limit'].fillna(cards['credit_limit'].median())

# --- Convert date fields to datetime ---
cards['acct_open_date'] = pd.to_datetime(cards['acct_open_date'], errors='coerce')
cards['expires'] = pd.to_datetime(cards['expires'], errors='coerce')




Standardize Data Types

In [74]:
# Convert categorical columns to category dtype
transactions['use_chip'] = transactions['use_chip'].astype('category')
cards['card_brand'] = cards['card_brand'].astype('category')
users['gender'] = users['gender'].astype('category')

# Ensure numeric columns are numeric
transactions['amount'] = pd.to_numeric(transactions['amount'], errors='coerce')
users['credit_score'] = pd.to_numeric(users['credit_score'], errors='coerce')
cards['credit_limit'] = pd.to_numeric(cards['credit_limit'], errors='coerce')


In [ ]:
transactions.to_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/clean_transactions.csv", index=False)
users.to_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/clean_users.csv", index=False)
cards.to_csv("/Users/deepakpatil/coffee-and-code/data_analyst/dataset/clean_cards.csv", index=False)
